In [1]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./bella_vista.txt")
docs = loader.load()

In [2]:
print(docs)
print(len(docs))

[Document(page_content="Q: What are the hours of operation for Bella Vista?\nA: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome guests from 12 p.m. to 10 p.m.\n\nQ: What type of cuisine does Bella Vista serve?\nA: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.\n\nQ: Do you offer vegetarian or vegan options at Bella Vista?\nA: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and vegan dishes. Our chefs are also happy to customize dishes based on dietary needs.\n\nQ: Is Bella Vista family-friendly?\nA: Yes, Bella Vista is a family-friendly establishment. We have a dedicated kids' menu and offer high chairs and booster seats for our younger guests.\n\nQ: Can I book private events at Bella Vista?\nA: Certainly! Bella Vista has a private dining area perfect for events, parties, or co

In [3]:
from langchain.schema import Document

example_doc = Document(page_content="test", metadata={"important_info": "hi there"})
example_doc

Document(page_content='test', metadata={'important_info': 'hi there'})

Texts are not loaded 1:1 into the database, but in pieces, so called "chunks". You can define the chunk size and the overlap between the chunks.

To create multiple documents (chunks), you can use a text splitter.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(docs)

In [5]:
for doc in documents:
    print(doc)

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='A: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='Q: What type of cuisine does Bella Vista serve?' metadata={'source': './bella_vista.txt'}
page_content='A: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We' metadata={'source': './bella_vista.txt'}
page_content='cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.' metadata={'source': './bella_vista.txt'}
page_content='Q: Do you offer vegetarian or vegan options at Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='A: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and veg

### Embeddings

Texts are not stored as text in the database, but as vector representations. 
Embeddings are a type of word representation that represents the semantic meaning of words in a vector space.

In [6]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [7]:
embedding1 = embeddings.embed_query(text="The solar system consists of the Sun and the objects that orbit it")
print(embedding1)
print(len(embedding1))

[0.024948249379936973, 0.02101100671047783, -0.01027025871649175, -0.02233580193630517, -0.04036292180969641, 0.02237294494432805, -0.0054291846910185316, -0.016306125111405733, -0.0011406239693415071, -0.009397380332824704, 0.00900117986534393, 0.005327039090649115, -0.01436226671334735, 0.0019887406969059454, -0.005989436703562784, -0.015464199234424417, 0.011174091469489732, -0.004144628189543944, 0.006357779449365098, -0.003819620117254906, 0.002332320738867723, -0.007007796059604475, -0.011465051551593816, -0.0002129964477250386, 0.01722233979840061, -0.004200344098562163, 0.011675532322347193, -0.031869375161192294, 0.0228062893511543, -0.009378807897490663, 0.02023098677819058, -0.010883131387385649, -0.019946218128746245, -0.015538487113115375, 0.01560039398913204, 0.00416010490854811, 0.007100655442306872, 0.010010253003718602, -0.03206747492927138, 0.004129151936201079, 0.011483623055605253, 0.03355322877780013, -0.00470797591841807, -0.005364183029994594, -0.0127774648434242

In [8]:
embedding2 = embeddings.embed_query(text="The solar system consists of the Sun and the objects that orbit it")
embedding3 = embeddings.embed_query(text="Planets, asteroids, and comets are part of our solar system.")
embedding4 = embeddings.embed_query(text="I love baking chocolate chip cookies on weekends.")

In [9]:
import numpy as np

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    return dot_product / (norm_a * norm_b)

In [10]:
sim_1_2 = cosine_similarity(embedding1, embedding2)
sim_1_4 = cosine_similarity(embedding1, embedding3)
sim_3_4 = cosine_similarity(embedding3, embedding4)

print(sim_1_2, sim_1_4, sim_3_4)


0.9999710384527462 0.9010602967017675 0.6977020989639189


### Loading Vectors into VectorDB (FAISS)

As created by OpenAIEmbeddings vectors and documents can now be stored in the database. This DB can be stored as .pkl file

In [11]:
from langchain.vectorstores.faiss import FAISS
import pickle

vectorstore = FAISS.from_documents(documents, embeddings)

with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

### Loading the database

Before using the database, it must of course be loaded again.

In [12]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)


In [20]:
retriever = vectorstore.as_retriever()

In [21]:
docs = retriever.get_relevant_documents(query="When are the opening hours??")
for doc in docs:
    print(doc)
    

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends' metadata={'source': './bella_vista.txt'}
page_content='during weekends and holidays, to ensure a seamless dining experience.' metadata={'source': './bella_vista.txt'}


In [23]:
docs = retriever.get_relevant_documents(query="When are the opening hours?", filter={'source': './bella_vista.txt'}, k=3)
for doc in docs:
    print(doc) # does not work!

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends' metadata={'source': './bella_vista.txt'}
page_content='during weekends and holidays, to ensure a seamless dining experience.' metadata={'source': './bella_vista.txt'}


In [25]:
retriever = vectorstore.as_retriever(search_kwargs={"filter": {'source': './bella_vista.txt'}, "k":1})
docs = retriever.get_relevant_documents(query="When are the opening hours??")
for doc in docs:
    print(doc)

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}


### Now we have to pass the documents to an LLM.

We create a prompt with a question and context. Context is the output from the retriever (Document Store).
LangChain provides chains out-of-the-box to do that, the RetrievalChains

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our restaurant.

{context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

llm = ChatOpenAI()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

result = qa.run(query="When are the opening hours??")
print(result)

The opening hours for Bella Vista are from 12 p.m.
